In [6]:
import re
import GTPyhop.gtpyhop as gtpyhop
import pddlgym

In [ ]:

def parse_pddl_problem(pddl_file_path):
    """
    Parse a PDDL problem file and extract relevant information.
    
    Args:
        pddl_file_path: Path to the PDDL problem file
        
    Returns:
        Dictionary containing parsed problem information
    """
    with open(pddl_file_path, 'r') as f:
        content = f.read()
    
    # Extract problem name
    problem_match = re.search(r'\(define\s+\(problem\s+([^\)]+)\)', content)
    problem_name = problem_match.group(1) if problem_match else "unknown"
    
    # Extract objects
    objects_match = re.search(r'\(:objects\s+(.*?)\)', content, re.DOTALL)
    objects = []
    if objects_match:
        objects = objects_match.group(1).strip().split()
    
    # Extract init statements
    init_match = re.search(r'\(:init\s+(.*?)\)\s*\(:goal', content, re.DOTALL)
    init_statements = []
    if init_match:
        init_content = init_match.group(1)
        # Find all predicates in init
        init_statements = re.findall(r'\([^\)]+\)', init_content)
    
    # Extract goal statements
    goal_match = re.search(r'\(:goal\s+\(and\s+(.*?)\)\s*\)\s*\)', content, re.DOTALL)
    goal_statements = []
    if goal_match:
        goal_content = goal_match.group(1)
        goal_statements = re.findall(r'\(not\s+\([^\)]+\)\)|\([^\)]+\)', goal_content)
    
    return {
        'problem_name': problem_name,
        'objects': objects,
        'init': init_statements,
        'goal': goal_statements
    }


def pddl_to_gtpyhop(pddl_file_path, problem_name="PDDLEnvSnake-v0"):
    """
    Convert a PDDL problem file to GTPyhop representation.
    
    Args:
        pddl_file_path: Path to the PDDL problem file
        problem_name: Name to use for the problem (default: "PDDLEnvSnake-v0")
        
    Returns:
        String containing the Python code for GTPyhop representation
    """
    parsed = parse_pddl_problem(pddl_file_path)
    
    # Determine grid size from objects
    coords = [obj for obj in parsed['objects'] if obj.startswith('pos')]
    max_x = max_y = 0
    for coord in coords:
        match = re.match(r'pos(\d+)-(\d+)', coord)
        if match:
            x, y = int(match.group(1)), int(match.group(2))
            max_x = max(max_x, x)
            max_y = max(max_y, y)
    size = max(max_x, max_y) + 1
    
    # Parse init statements
    isadjacent_pairs = []
    tailsnake = headsnake = spawn = None
    nextsnake_pairs = []
    nextspawn_pairs = []
    ispoint_list = []
    
    for stmt in parsed['init']:
        stmt = stmt.strip()
        if 'ISADJACENT' in stmt:
            match = re.search(r'ISADJACENT\s+(\S+)\s+(\S+)', stmt)
            if match:
                isadjacent_pairs.append((match.group(1), match.group(2)))
        elif 'tailsnake' in stmt:
            match = re.search(r'tailsnake\s+(\S+)', stmt)
            if match:
                tailsnake = match.group(1)
        elif 'headsnake' in stmt:
            match = re.search(r'headsnake\s+(\S+)', stmt)
            if match:
                headsnake = match.group(1)
        elif 'nextsnake' in stmt:
            match = re.search(r'nextsnake\s+(\S+)\s+(\S+)', stmt)
            if match:
                nextsnake_pairs.append((match.group(1), match.group(2)))
        elif 'spawn' in stmt and 'NEXTSPAWN' not in stmt:
            match = re.search(r'spawn\s+(\S+)', stmt)
            if match:
                spawn = match.group(1)
        elif 'NEXTSPAWN' in stmt:
            match = re.search(r'NEXTSPAWN\s+(\S+)\s+(\S+)', stmt)
            if match:
                nextspawn_pairs.append((match.group(1), match.group(2)))
        elif 'ispoint' in stmt:
            match = re.search(r'ispoint\s+(\S+)', stmt)
            if match:
                ispoint_list.append(match.group(1))
    

    rigid = gtpyhop.State('rigid relations')

    # snake domain is only coordinates/fields according to problem size
    # PDDL is a {size}x{size} grid, so coords are (0,0) to ({size-1},{size-1})

    # size = {size}
    coords = []
    for i in range(size):
        for j in range(size):
            coord = f'pos{{i}}-{{j}}'
            coords.append(coord)

    coords.append('dummypoint')

    rigid.types = {
        'coord': coords,
    }

    # define initial state for {problem_name}
    # if problem_name == "{problem_name}":
    initial = gtpyhop.State('state0')
    initial.grid_size = size

    # point adjacency relations
    isadjacent = {'dummypoint': []}

    for i in range(size):
        for j in range(size):
            coord = f'pos{{i}}-{{j}}'
            adjacent_coords = []
            # up
            if i > 0:
                adjacent_coords.append(f'pos{{i-1}}-{{j}}')
            # down
            if i < size - 1:
                adjacent_coords.append(f'pos{{i+1}}-{{j}}')
            # left
            if j > 0:
                adjacent_coords.append(f'pos{{i}}-{{j-1}}')
            # right
            if j < size - 1:
                adjacent_coords.append(f'pos{{i}}-{{j+1}}')
            isadjacent[coord] = adjacent_coords

    initial.isadjacent = isadjacent

    # snake spawn + point spawns
    # set initial tail position
    tailsnake  = '{tailsnake}'
    initial.tailsnake = {tailsnake: True} | {coord: False for coord in rigid.types['coord'] if coord != tailsnake}

    # set initial head position
    headsnake = '{headsnake}'
    initial.headsnake = {headsnake: True} | {coord: False for coord in rigid.types['coord'] if coord != headsnake}

    # initial nextsnake position
    nextsnake = {{', '.join(f"'{pair[0]}': '{pair[1]}'" for pair in nextsnake_pairs)}}
    initial.nextsnake = nextsnake | {coord: None for coord in rigid.types['coord'] if coord not in nextsnake.keys()}

    # set blocked positions (obstacles + snake spawnpoints)
    # NOTE: no obstacles in this problem
    blocked_pos = []
    blocked_pos.extend([tailsnake, headsnake])
    initial.blocked = {{coord: True for coord in blocked_pos}} | {{coord: False for coord in rigid.types['coord'] if coord not in blocked_pos}}
    
    # set initial food spawn
    spawn = '{spawn}'
    initial.spawn = {{spawn: True}} | {{coord: False for coord in rigid.types['coord'] if coord != spawn}}

    # nextspawn relations
    nextspawn = {{
        {',\n        '.join(f"'{pair[0]}': '{pair[1]}'" for pair in nextspawn_pairs)}
    }}

    initial.nextspawn = nextspawn | {{coord: None for coord in rigid.types['coord'] if coord not in nextspawn.keys()}}

    # ispoint
    ispoint = [{', '.join(f"'{point}'" for point in ispoint_list)}]
    initial.ispoint = {{coord: True for coord in ispoint}} | {{coord: False for coord in rigid.types['coord'] if coord not in ispoint}}

    # else:
    #     raise NotImplementedError("Only {problem_name} is implemented")
    
    return initial, rigid
        


In [6]:
pddl_to_gtpyhop('pddlgym/pddlgym/pddl/snake/p01.pddl')

TypeError: unhashable type: 'dict'